In [2]:
import pandas as pd
import warnings

In [4]:
pd.set_option('display.max_colwidth', None)  
pd.set_option('display.max_columns', None)  
pd.set_option('display.max_rows', 100)       
pd.set_option('display.precision', 2)
pd.options.display.max_seq_items = 1000

warnings.filterwarnings('ignore', category=UserWarning)

## DATASETS TO IMPORT

### Shipped Order Lines

In [50]:
df_lines = pd.read_csv('order_lines.csv', index_col=0)
print(f"{df_lines.shape[0]:,} order lines to process")
df_lines.head()

5,208 order lines to process


,Date,Month-Year,Warehouse Code,Customer Code,Order Number,Order Line Number,Item Code,Units,Euros
112,2021-01-04 00:00:00.000,1-2021,3403434,5002915-15,20247100,1,963543-43,2.0,381.47
14485,2021-01-07 00:00:00.000,1-2021,3403434,5397843-43,20258239,1,8902753-53,250.0,187.03
14486,2021-01-07 00:00:00.000,1-2021,3403434,5397843-43,20258239,2,8308591-91,500.0,452.45
14487,2021-01-07 00:00:00.000,1-2021,3403434,5397843-43,20258239,3,8308621-21,500.0,452.45
14488,2021-01-07 00:00:00.000,1-2021,3403434,5397843-43,20258239,4,8022921-21,400.0,567.16


### Distances

In [15]:
df_dist = pd.read_csv('distances.csv', index_col = 0)
df_dist['Location'] = df_dist['Customer Country'].astype(str) + ', ' + df_dist['Customer City'].astype(str)
df_dist.head()

,Warehouse Code,Warehouse Name,Warehouse Country,Warehouse City,Customer Code,Customer Country,Customer City,Road,Rail,Sea,Air,Location
19,3403434,WAREHOUSE PARIS AREA 1,FRANCE,CHALONS-EN-CHAMPAGNE,5002915-15,FRANCE,LES ANGLES,765.73,0.0,0.0,0.0,"FRANCE, LES ANGLES"
610,3403434,WAREHOUSE PARIS AREA 1,FRANCE,CHALONS-EN-CHAMPAGNE,5397843-43,FRANCE,SUCY EN BRIE,281.49,0.0,0.0,0.0,"FRANCE, SUCY EN BRIE"
676,3403434,WAREHOUSE PARIS AREA 1,FRANCE,CHALONS-EN-CHAMPAGNE,5340669-69,GERMANY,PEINE - WOLTORF,856.00,0.0,0.0,0.0,"GERMANY, PEINE - WOLTORF"
682,3403434,WAREHOUSE PARIS AREA 1,FRANCE,CHALONS-EN-CHAMPAGNE,5340672-72,FRANCE,BELLEVILLE,52.69,0.0,0.0,0.0,"FRANCE, BELLEVILLE"
1375,3403434,WAREHOUSE PARIS AREA 1,FRANCE,CHALONS-EN-CHAMPAGNE,5029913-13,FRANCE,MOISSY-CRAMAYEL,288.44,0.0,0.0,0.0,"FRANCE, MOISSY-CRAMAYEL"


### GPS Locations

In [22]:
df_gps = pd.read_csv('gps_locations.csv', index_col = 0)
print("{:,} Locations".format(len(df_gps)))
df_gps.head()

19 Locations


,Location,GPS 1,GPS 2
867,"BULGARIA, DOLNI BOGROV",42.70,23.49
1087,"FRANCE, AUBROMETZ",50.30,2.18
1153,"FRANCE, BELLEVILLE",48.87,2.39
1371,"FRANCE, CLICHY CEDEX",48.90,2.30
1534,"FRANCE, GAEL",48.12,-2.23


### UOM Conversions

In [37]:
df_uom = pd.read_csv('uom_conversions.csv', index_col = 0)
print("{:,} Unit of Measure Conversions".format(len(df_uom)))
df_uom.head()

557 Unit of Measure Conversions


,Item Code,Conversion Ratio
51,995097-97,0.02
100,995096-96,0.02
113,8949591-91,0.05
131,952547-47,0.04
276,866370-70,0.04


In [39]:
df_join = df_lines.copy()
COLS_JOIN = ['Item Code']
df_join = pd.merge(df_join, df_uom, on=COLS_JOIN, how='left', suffixes=('', '_y'))
df_join.drop(df_join.filter(regex='_y$').columns.tolist(),axis=1, inplace=True)
print("{:,} records".format(len(df_join)))
df_join.head()

5,208 records


,Date,Month-Year,Warehouse Code,Customer Code,Order Number,Order Line Number,Item Code,Units,Euros,Conversion Ratio
0,2021-01-04 00:00:00.000,1-2021,3403434,5002915-15,20247100,1,963543-43,2.0,381.47,56.58
1,2021-01-07 00:00:00.000,1-2021,3403434,5397843-43,20258239,1,8902753-53,250.0,187.03,0.03
2,2021-01-07 00:00:00.000,1-2021,3403434,5397843-43,20258239,2,8308591-91,500.0,452.45,0.02
3,2021-01-07 00:00:00.000,1-2021,3403434,5397843-43,20258239,3,8308621-21,500.0,452.45,0.02
4,2021-01-07 00:00:00.000,1-2021,3403434,5397843-43,20258239,4,8022921-21,400.0,567.16,0.03


## Data Processing

### Distance + GPD Locations

In [46]:
df_dist = pd.merge(df_dist, df_gps, on='Location', how='left', suffixes=('', '_y'))
df_dist.drop(df_dist.filter(regex='_y$').columns.tolist(),axis=1, inplace=True)
df_dist.head()

,Warehouse Code,Warehouse Name,Warehouse Country,Warehouse City,Customer Code,Customer Country,Customer City,Road,Rail,Sea,Air,Location,GPS 1,GPS 2
0,3403434,WAREHOUSE PARIS AREA 1,FRANCE,CHALONS-EN-CHAMPAGNE,5002915-15,FRANCE,LES ANGLES,765.73,0.0,0.0,0.0,"FRANCE, LES ANGLES",42.58,2.05
1,3403434,WAREHOUSE PARIS AREA 1,FRANCE,CHALONS-EN-CHAMPAGNE,5397843-43,FRANCE,SUCY EN BRIE,281.49,0.0,0.0,0.0,"FRANCE, SUCY EN BRIE",48.77,2.54
2,3403434,WAREHOUSE PARIS AREA 1,FRANCE,CHALONS-EN-CHAMPAGNE,5340669-69,GERMANY,PEINE - WOLTORF,856.00,0.0,0.0,0.0,"GERMANY, PEINE - WOLTORF",52.30,10.31
3,3403434,WAREHOUSE PARIS AREA 1,FRANCE,CHALONS-EN-CHAMPAGNE,5340672-72,FRANCE,BELLEVILLE,52.69,0.0,0.0,0.0,"FRANCE, BELLEVILLE",48.87,2.39
4,3403434,WAREHOUSE PARIS AREA 1,FRANCE,CHALONS-EN-CHAMPAGNE,5029913-13,FRANCE,MOISSY-CRAMAYEL,288.44,0.0,0.0,0.0,"FRANCE, MOISSY-CRAMAYEL",48.62,2.60


### Final Join

In [44]:
COLS_JOIN = ['Warehouse Code', 'Customer Code']
df_join = pd.merge(df_join, df_dist, on = COLS_JOIN, how='left', suffixes=('', '_y'))
df_join.drop(df_join.filter(regex='_y$').columns.tolist(),axis=1, inplace=True)
print("{:,} records".format(len(df_join)))
df_join.head()

5,208 records


,Date,Month-Year,Warehouse Code,Customer Code,Order Number,Order Line Number,Item Code,Units,Euros,Conversion Ratio,Warehouse Name,Warehouse Country,Warehouse City,Customer Country,Customer City,Road,Rail,Sea,Air,Location,GPS 1,GPS 2
0,2021-01-04 00:00:00.000,1-2021,3403434,5002915-15,20247100,1,963543-43,2.0,381.47,56.58,WAREHOUSE PARIS AREA 1,FRANCE,CHALONS-EN-CHAMPAGNE,FRANCE,LES ANGLES,765.73,0.0,0.0,0.0,"FRANCE, LES ANGLES",42.58,2.05
1,2021-01-07 00:00:00.000,1-2021,3403434,5397843-43,20258239,1,8902753-53,250.0,187.03,0.03,WAREHOUSE PARIS AREA 1,FRANCE,CHALONS-EN-CHAMPAGNE,FRANCE,SUCY EN BRIE,281.49,0.0,0.0,0.0,"FRANCE, SUCY EN BRIE",48.77,2.54
2,2021-01-07 00:00:00.000,1-2021,3403434,5397843-43,20258239,2,8308591-91,500.0,452.45,0.02,WAREHOUSE PARIS AREA 1,FRANCE,CHALONS-EN-CHAMPAGNE,FRANCE,SUCY EN BRIE,281.49,0.0,0.0,0.0,"FRANCE, SUCY EN BRIE",48.77,2.54
3,2021-01-07 00:00:00.000,1-2021,3403434,5397843-43,20258239,3,8308621-21,500.0,452.45,0.02,WAREHOUSE PARIS AREA 1,FRANCE,CHALONS-EN-CHAMPAGNE,FRANCE,SUCY EN BRIE,281.49,0.0,0.0,0.0,"FRANCE, SUCY EN BRIE",48.77,2.54
4,2021-01-07 00:00:00.000,1-2021,3403434,5397843-43,20258239,4,8022921-21,400.0,567.16,0.03,WAREHOUSE PARIS AREA 1,FRANCE,CHALONS-EN-CHAMPAGNE,FRANCE,SUCY EN BRIE,281.49,0.0,0.0,0.0,"FRANCE, SUCY EN BRIE",48.77,2.54


## CALCULATIONS

### Order Line Level

In [68]:
# Calculation @ line level
df_line = df_join.copy()

# Create the KG column
df_line['KG'] = df_line['Units'] * df_line['Conversion Ratio']

dict_co2e = dict(zip(['Air' ,'Sea', 'Road', 'Rail'], [2.1, 0.01, 0.096, 0.028]))
MODES = ['Road', 'Rail','Sea', 'Air']
for mode in MODES:
    df_line['CO2 ' + mode] = df_line['KG'].astype(float)/1000 * df_line[mode].astype(float) * dict_co2e[mode]
df_line['CO2 Total'] = df_line[['CO2 ' + mode for mode in MODES]].sum(axis = 1)
df_line.to_csv('detailed_report.csv')
df_line.head()

,Date,Month-Year,Warehouse Code,Customer Code,Order Number,Order Line Number,Item Code,Units,Euros,Conversion Ratio,Warehouse Name,Warehouse Country,Warehouse City,Customer Country,Customer City,Road,Rail,Sea,Air,Location,GPS 1,GPS 2,KG,CO2 Road,CO2 Rail,CO2 Sea,CO2 Air,CO2 Total
0,2021-01-04 00:00:00.000,1-2021,3403434,5002915-15,20247100,1,963543-43,2.0,381.47,56.58,WAREHOUSE PARIS AREA 1,FRANCE,CHALONS-EN-CHAMPAGNE,FRANCE,LES ANGLES,765.73,0.0,0.0,0.0,"FRANCE, LES ANGLES",42.58,2.05,113.17,8.32,0.0,0.0,0.0,8.32
1,2021-01-07 00:00:00.000,1-2021,3403434,5397843-43,20258239,1,8902753-53,250.0,187.03,0.03,WAREHOUSE PARIS AREA 1,FRANCE,CHALONS-EN-CHAMPAGNE,FRANCE,SUCY EN BRIE,281.49,0.0,0.0,0.0,"FRANCE, SUCY EN BRIE",48.77,2.54,8.00,0.22,0.0,0.0,0.0,0.22
2,2021-01-07 00:00:00.000,1-2021,3403434,5397843-43,20258239,2,8308591-91,500.0,452.45,0.02,WAREHOUSE PARIS AREA 1,FRANCE,CHALONS-EN-CHAMPAGNE,FRANCE,SUCY EN BRIE,281.49,0.0,0.0,0.0,"FRANCE, SUCY EN BRIE",48.77,2.54,11.10,0.30,0.0,0.0,0.0,0.30
3,2021-01-07 00:00:00.000,1-2021,3403434,5397843-43,20258239,3,8308621-21,500.0,452.45,0.02,WAREHOUSE PARIS AREA 1,FRANCE,CHALONS-EN-CHAMPAGNE,FRANCE,SUCY EN BRIE,281.49,0.0,0.0,0.0,"FRANCE, SUCY EN BRIE",48.77,2.54,11.10,0.30,0.0,0.0,0.0,0.30
4,2021-01-07 00:00:00.000,1-2021,3403434,5397843-43,20258239,4,8022921-21,400.0,567.16,0.03,WAREHOUSE PARIS AREA 1,FRANCE,CHALONS-EN-CHAMPAGNE,FRANCE,SUCY EN BRIE,281.49,0.0,0.0,0.0,"FRANCE, SUCY EN BRIE",48.77,2.54,12.80,0.35,0.0,0.0,0.0,0.35


### Order Level

In [71]:
# Calculate Weight (KG)
df_join['KG'] = df_join['Units'] * df_join['Conversion Ratio']

# Agg by order
GPBY_ORDER = ['Date', 'Month-Year', 
        'Warehouse Code', 'Warehouse Name', 'Warehouse Country', 'Warehouse City',
        'Customer Code', 'Customer Country', 'Customer City','Location', 'GPS 1', 'GPS 2', 
        'Road', 'Rail', 'Sea', 'Air',
        'Order Number']
df_agg = pd.DataFrame(df_join.groupby(GPBY_ORDER)[['Units', 'KG']].sum())
df_agg.reset_index(inplace = True)
df_agg.head()

,Date,Month-Year,Warehouse Code,Warehouse Name,Warehouse Country,Warehouse City,Customer Code,Customer Country,Customer City,Location,GPS 1,GPS 2,Road,Rail,Sea,Air,Order Number,Units,KG
0,2021-01-04 00:00:00.000,1-2021,3403434,WAREHOUSE PARIS AREA 1,FRANCE,CHALONS-EN-CHAMPAGNE,5002915-15,FRANCE,LES ANGLES,"FRANCE, LES ANGLES",42.58,2.05,765.73,0.0,0.0,0.0,20247100,2.0,113.17
1,2021-01-07 00:00:00.000,1-2021,3403434,WAREHOUSE PARIS AREA 1,FRANCE,CHALONS-EN-CHAMPAGNE,5340669-69,GERMANY,PEINE - WOLTORF,"GERMANY, PEINE - WOLTORF",52.30,10.31,856.00,0.0,0.0,0.0,20203388,153.0,2530.32
2,2021-01-07 00:00:00.000,1-2021,3403434,WAREHOUSE PARIS AREA 1,FRANCE,CHALONS-EN-CHAMPAGNE,5340672-72,FRANCE,BELLEVILLE,"FRANCE, BELLEVILLE",48.87,2.39,52.69,0.0,0.0,0.0,20203383,128.0,2116.87
3,2021-01-07 00:00:00.000,1-2021,3403434,WAREHOUSE PARIS AREA 1,FRANCE,CHALONS-EN-CHAMPAGNE,5397843-43,FRANCE,SUCY EN BRIE,"FRANCE, SUCY EN BRIE",48.77,2.54,281.49,0.0,0.0,0.0,20258239,2771.0,71.45
4,2021-01-08 00:00:00.000,1-2021,3403434,WAREHOUSE PARIS AREA 1,FRANCE,CHALONS-EN-CHAMPAGNE,5003451-51,FRANCE,LEVALLOIS PERRET,"FRANCE, LEVALLOIS PERRET",48.89,2.29,295.17,0.0,0.0,0.0,21000061,2125.0,111.95


### CALCULATE CO2 as f(KG, Ratios)

In [74]:
# CO2 Emissions
dict_co2e = dict(zip(['Air' ,'Sea', 'Road', 'Rail'], [2.1, 0.01, 0.096, 0.028]))
MODES = ['Road', 'Rail','Sea', 'Air']
for mode in MODES:
    df_agg['CO2 ' + mode] = df_agg['KG'].astype(float)/1000 * df_agg[mode].astype(float) * dict_co2e[mode]
df_agg['CO2 Total'] = df_agg[['CO2 ' + mode for mode in MODES]].sum(axis = 1)
df_agg.head()

,Date,Month-Year,Warehouse Code,Warehouse Name,Warehouse Country,Warehouse City,Customer Code,Customer Country,Customer City,Location,GPS 1,GPS 2,Road,Rail,Sea,Air,Order Number,Units,KG,CO2 Road,CO2 Rail,CO2 Sea,CO2 Air,CO2 Total
0,2021-01-04 00:00:00.000,1-2021,3403434,WAREHOUSE PARIS AREA 1,FRANCE,CHALONS-EN-CHAMPAGNE,5002915-15,FRANCE,LES ANGLES,"FRANCE, LES ANGLES",42.58,2.05,765.73,0.0,0.0,0.0,20247100,2.0,113.17,8.32,0.0,0.0,0.0,8.32
1,2021-01-07 00:00:00.000,1-2021,3403434,WAREHOUSE PARIS AREA 1,FRANCE,CHALONS-EN-CHAMPAGNE,5340669-69,GERMANY,PEINE - WOLTORF,"GERMANY, PEINE - WOLTORF",52.30,10.31,856.00,0.0,0.0,0.0,20203388,153.0,2530.32,207.93,0.0,0.0,0.0,207.93
2,2021-01-07 00:00:00.000,1-2021,3403434,WAREHOUSE PARIS AREA 1,FRANCE,CHALONS-EN-CHAMPAGNE,5340672-72,FRANCE,BELLEVILLE,"FRANCE, BELLEVILLE",48.87,2.39,52.69,0.0,0.0,0.0,20203383,128.0,2116.87,10.71,0.0,0.0,0.0,10.71
3,2021-01-07 00:00:00.000,1-2021,3403434,WAREHOUSE PARIS AREA 1,FRANCE,CHALONS-EN-CHAMPAGNE,5397843-43,FRANCE,SUCY EN BRIE,"FRANCE, SUCY EN BRIE",48.77,2.54,281.49,0.0,0.0,0.0,20258239,2771.0,71.45,1.93,0.0,0.0,0.0,1.93
4,2021-01-08 00:00:00.000,1-2021,3403434,WAREHOUSE PARIS AREA 1,FRANCE,CHALONS-EN-CHAMPAGNE,5003451-51,FRANCE,LEVALLOIS PERRET,"FRANCE, LEVALLOIS PERRET",48.89,2.29,295.17,0.0,0.0,0.0,21000061,2125.0,111.95,3.17,0.0,0.0,0.0,3.17


## VISUALISATION

In [78]:
# Mapping the delivery Mode
df_agg['Delivery Mode'] = df_agg[MODES].astype(float).apply(
    lambda t: [mode if t[mode]>0 else '-' for mode in MODES], axis = 1)
dict_map = dict(zip(df_agg['Delivery Mode'].astype(str).unique(), 
  [i.replace(", '-'",'').replace("'-'",'').replace("'",'') for i in df_agg['Delivery Mode'].astype(str).unique()]))
df_agg['Delivery Mode'] = df_agg['Delivery Mode'].astype(str).map(dict_map)
df_agg

,Date,Month-Year,Warehouse Code,Warehouse Name,Warehouse Country,Warehouse City,Customer Code,Customer Country,Customer City,Location,GPS 1,GPS 2,Road,Rail,Sea,Air,Order Number,Units,KG,CO2 Road,CO2 Rail,CO2 Sea,CO2 Air,CO2 Total,Delivery Mode
0,2021-01-04 00:00:00.000,1-2021,3403434,WAREHOUSE PARIS AREA 1,FRANCE,CHALONS-EN-CHAMPAGNE,5002915-15,FRANCE,LES ANGLES,"FRANCE, LES ANGLES",42.58,2.05,765.73,0.0,0.0,0.0,20247100,2.0,113.17,8.32,0.0,0.00,0.0,8.32,[Road]
1,2021-01-07 00:00:00.000,1-2021,3403434,WAREHOUSE PARIS AREA 1,FRANCE,CHALONS-EN-CHAMPAGNE,5340669-69,GERMANY,PEINE - WOLTORF,"GERMANY, PEINE - WOLTORF",52.30,10.31,856.00,0.0,0.0,0.0,20203388,153.0,2530.32,207.93,0.0,0.00,0.0,207.93,[Road]
2,2021-01-07 00:00:00.000,1-2021,3403434,WAREHOUSE PARIS AREA 1,FRANCE,CHALONS-EN-CHAMPAGNE,5340672-72,FRANCE,BELLEVILLE,"FRANCE, BELLEVILLE",48.87,2.39,52.69,0.0,0.0,0.0,20203383,128.0,2116.87,10.71,0.0,0.00,0.0,10.71,[Road]
3,2021-01-07 00:00:00.000,1-2021,3403434,WAREHOUSE PARIS AREA 1,FRANCE,CHALONS-EN-CHAMPAGNE,5397843-43,FRANCE,SUCY EN BRIE,"FRANCE, SUCY EN BRIE",48.77,2.54,281.49,0.0,0.0,0.0,20258239,2771.0,71.45,1.93,0.0,0.00,0.0,1.93,[Road]
4,2021-01-08 00:00:00.000,1-2021,3403434,WAREHOUSE PARIS AREA 1,FRANCE,CHALONS-EN-CHAMPAGNE,5003451-51,FRANCE,LEVALLOIS PERRET,"FRANCE, LEVALLOIS PERRET",48.89,2.29,295.17,0.0,0.0,0.0,21000061,2125.0,111.95,3.17,0.0,0.00,0.0,3.17,[Road]
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
286,2021-10-26 00:00:00.000,10-2021,3403434,WAREHOUSE PARIS AREA 1,FRANCE,CHALONS-EN-CHAMPAGNE,5340672-72,FRANCE,BELLEVILLE,"FRANCE, BELLEVILLE",48.87,2.39,52.69,0.0,0.0,0.0,21004398,9040.0,416.75,2.11,0.0,0.00,0.0,2.11,[Road]
287,2021-10-27 00:00:00.000,10-2021,3403434,WAREHOUSE PARIS AREA 1,FRANCE,CHALONS-EN-CHAMPAGNE,5029913-13,FRANCE,MOISSY-CRAMAYEL,"FRANCE, MOISSY-CRAMAYEL",48.62,2.60,288.44,0.0,0.0,0.0,21089663,13515.0,493.52,13.67,0.0,0.00,0.0,13.67,[Road]
288,2021-10-27 00:00:00.000,10-2021,3403434,WAREHOUSE PARIS AREA 1,FRANCE,CHALONS-EN-CHAMPAGNE,5334855-55,UNITED KINGDOM,KENT,"UNITED KINGDOM, KENT",51.20,0.74,409.00,0.0,44.0,0.0,21009276,450.0,35.35,1.39,0.0,0.02,0.0,1.40,"[Road, Sea]"
289,2021-10-28 00:00:00.000,10-2021,3403434,WAREHOUSE PARIS AREA 1,FRANCE,CHALONS-EN-CHAMPAGNE,5397843-43,FRANCE,SUCY EN BRIE,"FRANCE, SUCY EN BRIE",48.77,2.54,281.49,0.0,0.0,0.0,21089497,11831.0,367.71,9.94,0.0,0.00,0.0,9.94,[Road]


In [80]:
# Save Final Report
df_agg.to_csv('final_report.csv')